In [ ]:
from lxml import html
import lxml
import time
import requests
import pandas as pd
import sqlite3

In [ ]:
date_jour = '2017-12-31'
OUTPUT_PATH = '../_outputs/'
database_file = OUTPUT_PATH + 'chefs_{}.db'.format(date_jour)
conn = sqlite3.connect(database_file)

In [ ]:
df_source = pd.read_sql_query("select * from index_chefs;", conn)
df_source.head()

In [ ]:
def getChefPage(chef_id):
    url = 'https://labelleassiette.fr/' + chef_id
    print("Page : " + url)
    response = requests.get(url)
    content_source = response.text
    return [chef_id, html.document_fromstring(content_source)]

In [ ]:
docs = []
for index, row in df_source.iterrows():
    docs.append(getChefPage(row['chef']))

In [ ]:
len(docs)

In [ ]:
docs[0][1]

In [ ]:
def get_rows(doc):
    ratings = doc.xpath('//div[@class="feedback"]//span[@itemprop="ratingValue"]/@content')
    authors = doc.xpath('//div[@class="feedback"]//span[@itemprop="author"]/@content')
    authors_types = doc.xpath('//div[@class="feedback"]//span[2]/text()')
    dates = doc.xpath('//div[@class="feedback"]//span[@itemprop="datePublished"]/@content')
    descriptions = doc.xpath('//div[@class="feedback"]//p[@itemprop="description"]')
    
    return ratings, authors, authors_types, dates, descriptions
    # return len(ratings), len(authors), len(authors_types), len(dates), len(descriptions)

In [ ]:
# get_rows(docs[9][1])

In [ ]:
docs[36][0]

In [ ]:
columns = ['chef', 'rating', 'author', 'author_type', 'dates', 'description']
df_output = pd.DataFrame(columns=columns)

In [ ]:
idx = 0
for doc in docs:
    ratings, authors, authors_types, dates, descriptions = get_rows(doc[1])
    chef = doc[0]
    # print(chef)
    idxm = 0
    for r in ratings:
        descriptions[idxm] = descriptions[idxm].text_content()
        df_output.loc[idx] = [chef, r, authors[idxm], authors_types[idxm], dates[idxm], descriptions[idxm]]
        idx +=1
        idxm +=1

In [ ]:
df_output.head(50)

In [ ]:
df_output['description'][30]

In [ ]:
df_output.to_excel(OUTPUT_PATH + 'chefs_{}_avis.xlsx'.format(date_jour), index=False, header=True)